In [108]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/sa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [109]:
import os
import getpass

EMBEDDING_MODEL = "text-embedding-3-small"
CHAT_MODEL = "gpt-4o-mini"   # adjust as desired
COLLECTION_NAME = "adn"
TOP_K = 6
FETCH_K = 24

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [110]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta
from langchain_core.documents import Document
import csv
from typing import List

def load_csv_with_csvloader(csv_path: str) -> List[Document]:
    """
    Load a CSV into Documents using LangChain's CSVLoader.
    page_content := only ['title', 'details'].
    All remaining columns preserved as metadata (e.g., id, section, tags, channel_number).
    """
    with open(csv_path, newline="", encoding="utf-8-sig") as f:
        cols = csv.DictReader(f).fieldnames or []

    content_cols = [c for c in ["title", "details"] if c in cols]
    meta_cols = [c for c in cols if c not in content_cols]
    print(f"content_cols: {content_cols}")
    print(f"meta_cols: {meta_cols}")
    loader = CSVLoader(
        file_path=csv_path,
        content_columns=content_cols,
        metadata_columns=meta_cols,
        encoding="utf-8-sig",
        autodetect_encoding=True,
    )
    return loader.load()

# current metadata columns: id,section,subsection,title,details,price_crc,price_text,tags,url,contact_value,channel_number,locale,version
loader = CSVLoader(
    file_path=f"./RAG_data/adn_rag_base_full_v1_3.csv",
    metadata_columns=[
      "id",
      "section",
      "subsection",
      "title",
      "details",
      "price_crc",
      "price_text",
      "tags",
      "url",
      "contact_value",
      "channel_number",
      "locale",
      "version"
    ]
)



In [111]:
adn_data = load_csv_with_csvloader("RAG_data/adn_rag_base_full_v1_3.csv")

for doc in adn_data:
    print(doc.page_content)
    print(doc.metadata)
    break

content_cols: ['title', 'details']
meta_cols: ['id', 'section', 'subsection', 'price_crc', 'price_text', 'tags', 'url', 'contact_value', 'channel_number', 'locale', 'version']
title: Descripción
details: American Data Networks S.A. (ADN) fundada en 2005. Equipo con más de 15 años de experiencia en telecomunicaciones.
Enfoque 100 % en calidad de servicio y soporte oportuno. Actualización constante de tecnologías para brindar servicios de clase mundial.
Opción de transporte nacional e internacional de alta capacidad de datos en Costa Rica; permite optimizar y expandir redes de forma segura y rápida.
{'source': 'RAG_data/adn_rag_base_full_v1_3.csv', 'row': 0, 'id': '2eebd6ef-cd3e-46e4-a268-dd4830bf76aa', 'section': 'Compañía', 'subsection': 'Quiénes Somos', 'price_crc': '', 'price_text': '', 'tags': 'empresa, historia, calidad, telecomunicaciones', 'url': '', 'contact_value': '', 'channel_number': '', 'locale': 'es_CR', 'version': 'v1.3'}


In [112]:
adn_data[0]

Document(metadata={'source': 'RAG_data/adn_rag_base_full_v1_3.csv', 'row': 0, 'id': '2eebd6ef-cd3e-46e4-a268-dd4830bf76aa', 'section': 'Compañía', 'subsection': 'Quiénes Somos', 'price_crc': '', 'price_text': '', 'tags': 'empresa, historia, calidad, telecomunicaciones', 'url': '', 'contact_value': '', 'channel_number': '', 'locale': 'es_CR', 'version': 'v1.3'}, page_content='title: Descripción\ndetails: American Data Networks S.A. (ADN) fundada en 2005. Equipo con más de 15 años de experiencia en telecomunicaciones.\nEnfoque 100 % en calidad de servicio y soporte oportuno. Actualización constante de tecnologías para brindar servicios de clase mundial.\nOpción de transporte nacional e internacional de alta capacidad de datos en Costa Rica; permite optimizar y expandir redes de forma segura y rápida.')

In [113]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    adn_data,
    embeddings,
    location=":memory:",
    collection_name=COLLECTION_NAME
)



In [114]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

semantic_documents = semantic_chunker.split_documents(adn_data)

semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="ADN_Data_Semantic_Chunks"
)

In [115]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 6})

semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 6})



In [116]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
Eres un asistente de soporte que trabaja para American Data Networks. Habla siempre en primera persona como si fueras parte del equipo. Responde SOLO con la información del CONTEXTO proporcionado.
Si la pregunta no se encuentra en el contexto proporcionado de American Data Networks, responde: "No tengo la respuesta a esa pregunta".
Si la pregunta es sobre un producto o servicio que no es de American Data Networks, responde: "No tenemos información sobre ese producto o servicio".

Responde en el mismo idioma en que se hizo la pregunta.
Importante: usa expresiones en primera persona (por ejemplo: “nuestra dirección”, “puedes visitarnos”, “podemos ayudarte”), nunca en tercera persona (“ellos”, “tienen”, “puedes visitarlos”).

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [117]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

In [118]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [119]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [120]:
naive_retrieval_chain.invoke({"question" : "tienen History Channel?"})["response"].content

'Sí, en nuestra grilla IPTV tenemos disponible el canal History Channel.'

In [121]:
semantic_retrieval_chain.invoke({"question" : "Quiero contratar un servicio de ADN"})["response"].content

'Puedes contratar un servicio de ADN en línea siguiendo estos pasos: primero, verifica la cobertura en nuestro sitio https://data.cr/cobertura o comparte tu dirección exacta por WhatsApp. Luego, puedes elegir tu plan y adicionales, completar el formulario de checkout con tus datos personales y ubicación, y agenda la instalación en la fecha y hora que prefieras. También puedes contratar por WhatsApp, donde un asesor verificará tu cobertura, te recomendará un plan y te ayudará a registrar tus datos para finalizar la contratación. Si deseas más información o quieres iniciar el proceso, puedes visitarnos en https://data.cr/adquirir-servicios o contactarnos por WhatsApp.'

In [ ]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(adn_data,)

bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)
bm25_retrieval_chain.invoke({"question" : "Precio de un plan de 100/100 Mbps"})["response"].content

In [134]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

multi_query_retrieval_chain.invoke({"question" : "Precio de un plan de 100 megas, y que mas tienes, y cuales son los planes mas populares"})["response"].content

'El precio de un plan de 100 megas en nuestra compañía es de ₡20,500 IVI. Además, tenemos otros planes como el de 250/250 Mbps por ₡26,600 IVI, y el de 500/500 Mbps por ₡33,410 IVI. Nuestro plan más popular es el de 500/500 Mbps, que además incluye equipo Wi-Fi, firewall y soporte 24/7. También contamos con el plan de 1 Gbps por ₡49,500 IVI, que es el de mayor velocidad que ofrecemos. Si quieres más detalles o deseas contratar, puedes hacerlo en nuestra plataforma en línea o visitándonos con tu cobertura verificada.'

In [135]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = adn_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

parent_document_retriever.add_documents(parent_docs, ids=None)

parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

parent_document_retrieval_chain.invoke({"question" : "Precio de un plan de 100 megas, y que mas tienes, y cual es el plan mas popular"})["response"].content

'El precio del plan de 100 megas en American Data Networks es de ₡20,500 IVI. Además, contamos con otros planes como el de 250 Mbps, que cuesta ₡26,600 IVI, y el de 500 Mbps, que tiene un precio de ₡33,410 IVI. También ofrecemos un plan de 1 Gbps con un precio de ₡49,500 IVI.\n\nNuestro plan más popular es el de 500/500 Mbps, que incluye Equipo Wi-Fi, Firewall y Soporte 24/7.'

In [122]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [ ]:
from ragas.testset import TestsetGenerator
from typing import List

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

# Create manual test set with realistic user questions
manual_test_set_updated = [
    {
        "question": "¿Qué es American Data Networks (ADN) y desde cuándo existe?",
        "contexts": "",
        "answer": "",
        "ground_truth": "American Data Networks S.A. (ADN) es una empresa de telecomunicaciones fundada en 2005."
    },
    # {
    #     "question": "¿Cuál es la misión de ADN?",
    #     "contexts": "",
    #     "answer": "",
    #     "ground_truth": "Ser la primera opción en servicios de telecomunicaciones, garantizando transporte de datos rápido, eficiente y seguro."
    # },
    {
        "question": "¿Y la visión de la empresa cuál es?",
        "contexts": "",
        "answer": "",
        "ground_truth": "Ser la compañía líder en vanguardia tecnológica en Costa Rica, con enfoque claro en clientes y estándares de clase mundial."
    },
    {
        "question": "Estoy armando mi plan en casa: ¿cuánto cuesta el plan residencial de 100/100 megas?",
        "contexts": "",
        "answer": "",
        "ground_truth": "₡20 500 IVI."
    },
    # {
    #     "question": "¿Cuál es el precio del plan de 250/250 megas simétricos?",
    #     "contexts": "",
    #     "answer": "",
    #     "ground_truth": "₡26 600 IVI."
    # },
    {
        "question": "Quiero más velocidad: ¿cuánto vale el plan de 500/500 megas?",
        "contexts": "",
        "answer": "",
        "ground_truth": "₡33 410 IVI."
    },
    {
        "question": "¿Tienen plan de 1 giga simétrico? ¿En cuánto sale al mes?",
        "contexts": "",
        "answer": "",
        "ground_truth": "₡49 500 IVI por mes."
    },
    {
        "question": "¿Qué incluyen los planes residenciales además del Internet?",
        "contexts": "",
        "answer": "",
        "ground_truth": "Incluyen equipo Wi-Fi, firewall gestionado y soporte técnico 24/7."
    },
    {
        "question": "Quiero agregar telefonía fija a mi plan: ¿cuánto cuesta al mes?",
        "contexts": "",
        "answer": "",
        "ground_truth": "₡3 590 IVI/mes."
    },
    {
        "question": "Si compro IPTV, ¿cuántos dispositivos como máximo puedo adquirir?",
        "contexts": "",
        "answer": "",
        "ground_truth": "Hasta 10 dispositivos IPTV."
    },
    {
        "question": "¿Y cuánto cuesta cada dispositivo IPTV?",
        "contexts": "",
        "answer": "",
        "ground_truth": "₡4 590 IVI por dispositivo."
    },
    {
        "question": "Más o menos, ¿cuántos canales trae el servicio de IPTV?",
        "contexts": "",
        "answer": "",
        "ground_truth": "Más de 100 canales."
    },
    # {
    #     "question": "¿Qué impuestos y cargos ya vienen incluidos en los precios IVI?",
    #     "contexts": "",
    #     "answer": "",
    #     "ground_truth": "IVA 13 %, 1 % Cruz Roja y 0,75 % 9-1-1."
    # },
    # {
    #     "question": "¿Cuál es el número de WhatsApp para comprar o pedir soporte?",
    #     "contexts": "",
    #     "answer": "",
    #     "ground_truth": "+506 7087-8240."
    # },
    # {
    #     "question": "Necesito llamarles: ¿cuál es el teléfono principal de ADN?",
    #     "contexts": "",
    #     "answer": "",
    #     "ground_truth": "+506 4050-5050."
    # }
]


print("Generating test data for NAIVE retriever...")
naive_rag_responses = []

for item in manual_test_set_updated:
    question = item["question"]
    
    # Use naive retrieval chain
    response = naive_retrieval_chain.invoke({"question": question})
    
    updated_item = {
        "question": question,
        "contexts": [doc.page_content for doc in response['context']],
        "answer": response['response'].content,
        "ground_truth": item["ground_truth"]
    }
    
    naive_rag_responses.append(updated_item)
    print(f"✓ Processed: {question[:50]}...")




Generating test data for NAIVE retriever...
✓ Processed: ¿Qué es American Data Networks (ADN) y desde cuánd...
✓ Processed: ¿Cuál es la misión de ADN?...
✓ Processed: ¿Y la visión de la empresa cuál es?...
✓ Processed: Estoy armando mi plan en casa: ¿cuánto cuesta el p...
✓ Processed: ¿Cuál es el precio del plan de 250/250 megas simét...
✓ Processed: Quiero más velocidad: ¿cuánto vale el plan de 500/...
✓ Processed: ¿Tienen plan de 1 giga simétrico? ¿En cuánto sale ...
✓ Processed: ¿Qué incluyen los planes residenciales además del ...
✓ Processed: Quiero agregar telefonía fija a mi plan: ¿cuánto c...
✓ Processed: Si compro IPTV, ¿cuántos dispositivos como máximo ...
✓ Processed: ¿Y cuánto cuesta cada dispositivo IPTV?...
✓ Processed: Más o menos, ¿cuántos canales trae el servicio de ...
✓ Processed: ¿Qué impuestos y cargos ya vienen incluidos en los...
✓ Processed: ¿Cuál es el número de WhatsApp para comprar o pedi...
✓ Processed: Necesito llamarles: ¿cuál es el teléfono principal...



In [ ]:
print("\nGenerating test data for SEMANTIC retriever...")
semantic_rag_responses = []

for item in manual_test_set_updated:
    question = item["question"]
    
    # Use semantic retrieval chain
    response = semantic_retrieval_chain.invoke({"question": question})
    
    updated_item = {
        "question": question,
        "contexts": [doc.page_content for doc in response['context']],
        "answer": response['response'].content,
        "ground_truth": item["ground_truth"]
    }
    
    semantic_rag_responses.append(updated_item)
    print(f"✓ Processed: {question[:50]}...")

In [124]:
# Convert to RAGAS Dataset
from datasets import Dataset
import pandas as pd

# Create datasets for both retrievers
naive_df = pd.DataFrame(naive_rag_responses)
naive_dataset = Dataset.from_pandas(naive_df)

semantic_df = pd.DataFrame(semantic_rag_responses)
semantic_dataset = Dataset.from_pandas(semantic_df)

print(f"Naive dataset shape: {naive_df.shape}")
print(f"Semantic dataset shape: {semantic_df.shape}")


Naive dataset shape: (15, 4)
Semantic dataset shape: (15, 4)


In [125]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# Define metrics
metrics = [
    faithfulness,
    answer_relevancy, 
    context_precision,
    context_recall
]

print("Evaluating NAIVE retriever...")
naive_result = evaluate(
    naive_dataset,
    metrics=metrics,
    llm=generator_llm,
    embeddings=generator_embeddings
)

print("Evaluating SEMANTIC retriever...")
semantic_result = evaluate(
    semantic_dataset,
    metrics=metrics,
    llm=generator_llm,
    embeddings=generator_embeddings
)

print("✓ Both evaluations completed!")
semantic_result

Evaluating NAIVE retriever...


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Exception raised in Job[2]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[22]: TimeoutError()


Evaluating SEMANTIC retriever...


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Exception raised in Job[16]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1 in organization org-gs3dRZtJA0QKE5tZXAw4L9YF on tokens per min (TPM): Limit 30000, Used 29484, Requested 1255. Please try again in 1.478s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[6]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1 in organization org-gs3dRZtJA0QKE5tZXAw4L9YF on tokens per min (TPM): Limit 30000, Used 30000, Requested 1080. Please try again in 2.16s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
Exception raised in Job[0]: TimeoutError()
Exception raised in Job[12]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Exception raised in Job[20]: RateLimitError(Error code: 429 - {'error': {'message': 'Ra

✓ Both evaluations completed!


{'faithfulness': 0.8000, 'answer_relevancy': 0.9285, 'context_precision': 0.6191, 'context_recall': 0.6154}

In [126]:
naive_result

{'faithfulness': 0.8115, 'answer_relevancy': 0.9383, 'context_precision': 0.5358, 'context_recall': 0.5333}

In [127]:
import pandas as pd
import numpy as np

def extract_score(result, metric_name):
    """Safely extract score from RAGAS result."""
    try:
        score = result[metric_name]
        if isinstance(score, list):
            # Calculate average for list of scores
            valid_scores = [s for s in score if s is not None and not np.isnan(s)]
            return sum(valid_scores) / len(valid_scores) if valid_scores else 0.0
        elif isinstance(score, (int, float)) and not np.isnan(score):
            return float(score)
        else:
            return 0.0
    except (KeyError, TypeError):
        return 0.0

# Extract scores for both retrievers
metrics_names = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']
naive_scores = [extract_score(naive_result, metric) for metric in metrics_names]
semantic_scores = [extract_score(semantic_result, metric) for metric in metrics_names]

# Create comparison DataFrame
comparison_data = {
    'Metric': [
        'Faithfulness',
        'Answer Relevancy', 
        'Context Precision',
        'Context Recall'
    ],
    'Naive Retriever': naive_scores,
    'Semantic Retriever': semantic_scores,
    'Improvement': [semantic - naive for semantic, naive in zip(semantic_scores, naive_scores)],
    'Description': [
        'Factual consistency with retrieved context',
        'Relevance of answer to the question',
        'Precision of retrieved context',
        'Recall of relevant context'
    ]
}

comparison_df = pd.DataFrame(comparison_data)

# Calculate averages
naive_avg = np.mean([score for score in naive_scores if score > 0])
semantic_avg = np.mean([score for score in semantic_scores if score > 0])

print("=" * 80)
print("           RAG RETRIEVER PERFORMANCE COMPARISON")
print("=" * 80)
print()
print(comparison_df.round(4).to_string(index=False, justify='center'))
print()
print("-" * 80)
print(f"{'OVERALL AVERAGES':^80}")
print("-" * 80)
print(f"Naive Retriever Average:    {naive_avg:.4f}")
print(f"Semantic Retriever Average: {semantic_avg:.4f}")
print(f"Overall Improvement:        {semantic_avg - naive_avg:+.4f}")
print()

# Determine winner
if semantic_avg > naive_avg:
    winner = "🏆 SEMANTIC RETRIEVER WINS!"
    improvement_pct = ((semantic_avg - naive_avg) / naive_avg) * 100
    print(f"{winner:^80}")
    print(f"Performance improvement: +{improvement_pct:.1f}%")
else:
    winner = "🏆 NAIVE RETRIEVER WINS!"
    improvement_pct = ((naive_avg - semantic_avg) / semantic_avg) * 100
    print(f"{winner:^80}")
    print(f"Performance advantage: +{improvement_pct:.1f}%")

print("=" * 80)

           RAG RETRIEVER PERFORMANCE COMPARISON

      Metric       Naive Retriever  Semantic Retriever  Improvement                Description                
     Faithfulness      0.8115             0.8000         -0.0115    Factual consistency with retrieved context
 Answer Relevancy      0.9383             0.9285         -0.0098           Relevance of answer to the question
Context Precision      0.5358             0.6191          0.0833                Precision of retrieved context
   Context Recall      0.5333             0.6154          0.0821                    Recall of relevant context

--------------------------------------------------------------------------------
                                OVERALL AVERAGES                                
--------------------------------------------------------------------------------
Naive Retriever Average:    0.7047
Semantic Retriever Average: 0.7407
Overall Improvement:        +0.0360

                           🏆 SEMANTIC RETRIEV

In [128]:
# Detailed breakdown for better insights
print("\n" + "=" * 60)
print("             DETAILED ANALYSIS")
print("=" * 60)

# Check which retriever wins in each metric
metric_winners = []
for i, metric in enumerate(comparison_df['Metric']):
    naive_score = comparison_df.iloc[i]['Naive Retriever']
    semantic_score = comparison_df.iloc[i]['Semantic Retriever']
    improvement = comparison_df.iloc[i]['Improvement']
    
    if semantic_score > naive_score:
        winner = "Semantic"
        symbol = "📈"
    elif naive_score > semantic_score:
        winner = "Naive"
        symbol = "📉"
    else:
        winner = "Tie"
        symbol = "🔄"
    
    metric_winners.append(winner)
    print(f"{symbol} {metric}: {winner} wins (Δ: {improvement:+.4f})")

print()

# Performance insights
print("PERFORMANCE INSIGHTS:")
print("-" * 30)

# Best performing metrics for each
naive_best = comparison_df.loc[comparison_df['Naive Retriever'].idxmax(), 'Metric']
semantic_best = comparison_df.loc[comparison_df['Semantic Retriever'].idxmax(), 'Metric']

print(f"• Naive Retriever excels at: {naive_best}")
print(f"• Semantic Retriever excels at: {semantic_best}")

# Areas needing improvement
naive_worst = comparison_df.loc[comparison_df['Naive Retriever'].idxmin(), 'Metric']
semantic_worst = comparison_df.loc[comparison_df['Semantic Retriever'].idxmin(), 'Metric']

print(f"• Naive Retriever needs work on: {naive_worst}")
print(f"• Semantic Retriever needs work on: {semantic_worst}")

print("\nRECOMMENDations:")
print("-" * 20)
if semantic_avg > naive_avg:
    print("✅ Use SEMANTIC retriever for production")
    print("✅ Focus on improving Context Precision/Recall")
else:
    print("✅ Stick with NAIVE retriever for now")
    print("✅ Consider tuning semantic chunking parameters")

print("\n" + "=" * 60)


             DETAILED ANALYSIS
📉 Faithfulness: Naive wins (Δ: -0.0115)
📉 Answer Relevancy: Naive wins (Δ: -0.0098)
📈 Context Precision: Semantic wins (Δ: +0.0833)
📈 Context Recall: Semantic wins (Δ: +0.0821)

PERFORMANCE INSIGHTS:
------------------------------
• Naive Retriever excels at: Answer Relevancy
• Semantic Retriever excels at: Answer Relevancy
• Naive Retriever needs work on: Context Recall
• Semantic Retriever needs work on: Context Recall

RECOMMENDations:
--------------------
✅ Use SEMANTIC retriever for production
✅ Focus on improving Context Precision/Recall

